In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/')

In [2]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools

from model.GRU_model import *
from pipeline_config import *
from utils import *

In [3]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)
#torch.use_deterministic_algorithms(True)


In [4]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

In [5]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.00019837551214953915
Epochs : 100


In [6]:
cnt = 0
for indent in sorted(list(categories_per_indent_dict.keys())):
    for category in categories_per_indent_dict[indent]:
        cnt+=1

cnt

52

In [7]:
def hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path):

    categories_counter = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            print(f'indent: {indent}')
            category_name = category_id_to_name_dict[category]
            print(f'category_name: {category_name}')

            categories_counter+= 1
            print(f'categories_counter: {categories_counter}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                print(f'category with 0 loss coef: {category_name}|{category}')
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                loss_coef =  5.336535902170534*np.exp(-7)
                parent_model = Model
                parent_optimizer = Optimizer
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)
 
            
            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = Model
            optimizer = Optimizer
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            training_and_evaluation(model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
                


In [8]:
hgru_models = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath)

indent: 0
category_name: All-items
categories_counter: 1
category with 0 loss coef: All-items|0
indent: 1
category_name: Alcoholic beverages and tobacco
categories_counter: 2
indent: 1
category_name: Clothing and footwear
categories_counter: 3
indent: 1
category_name: Communications
categories_counter: 4
indent: 1
category_name: Education
categories_counter: 5
indent: 1
category_name: Food and non-alcoholic beverages
categories_counter: 6
indent: 1
category_name: Furnishings, household equipment and routine maintenance
categories_counter: 7
indent: 1
category_name: Health
categories_counter: 8
indent: 1
category_name: Housing, water, electricity, gas and other fuels
categories_counter: 9
indent: 1
category_name: Miscellaneous goods and services
categories_counter: 10
indent: 1
category_name: Recreation and culture
categories_counter: 11
indent: 1
category_name: Restaurants and hotels
categories_counter: 12
indent: 1
category_name: Transport
categories_counter: 13
indent: 2
category_nam

Save prediction dict:

In [9]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model = Model
        basic_optimizer = Optimizer
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [10]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])
    
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

In [11]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Save model weights in dict for bi-directional use:

In [12]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])

In [13]:
def get_weights_per_category(category_id_list, dir_path):
    basic_model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
    #basic_model.to(device)

    best_models_weights_dict = {}

    for category_id in category_id_list:
        category_name = category_id_to_name_dict[category_id]
        ckp_path = dir_path+category_name+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        category_model_weights = unify_model_weights(model)
        best_models_weights_dict[category_id] = category_model_weights
        
    return best_models_weights_dict

In [14]:
dir_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/hgru/models_weights/'

In [15]:
category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])


In [16]:
len(categories)

52

In [17]:
weights_dict = get_weights_per_category(category_id_list, dir_path)

In [18]:
with open(hgru_model_weights_path, 'wb') as handle:
    pickle.dump(weights_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
hgru_model_weights_path

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/pickle_files/norway_hgru_model_weights.pickle'